In [1]:
list1=[1,2,3,4,5,6,7,8,9]
list1.pop(3)

4

In [1]:
print("Hello {1!s} and {0!s}".format('Himanshu', 'Tarun'))

Hello Tarun and Himanshu


In [2]:
DEBUG = True


def GET_FAISS_RESOURCES():
    return None


def GET_FAISS_INDEX(index_path):
    #raise NotImplementedError
    import faiss
    index = faiss.read_index(index_path)
    return index


def GET_FAISS_ID_TO_VECTOR(ids_vectors_path):
    #raise NotImplementedError
    import pickle
    import os
    if not os.path.exists(ids_vectors_path):
      return None
    with open(ids_vectors_path, 'rb') as f:
      index_dict = pickle.load(f)
    def id_to_vector(id_):
      try:
        return index_dict[id_]
      except:
        pass
    return id_to_vector


UPDATE_FAISS_AFTER_SECONDS = None

IMAGESEARCH_TMP = r"D:/Kaggle/CleanCodeCup/2023/2023/images/images/"

# --------------------- Feature Detect
# resize size
NOR_X = 512
NOR_Y = 384

# phash size
PHASH_X = 8
PHASH_Y = 8
#PHASH_Y = 4

SIFT_DIMENSIONS = 128

# feature's count extracted from each image
NUM_FEATURES = 100
isAddPhash = False

# BoW
bow_num_words = 1000
dictionary_path = '/faiss-web-service/resources/dictionary'
# NUM_FEATURES = 0
# isAddPhash = False

# --------------------- Train
#INDEX_KEY = "IDMap,Flat"
INDEX_KEY = "IDMap,PCA128,IVF2048,PQ16"
#INDEX_KEY = "IDMap,OPQ16_64,IMI2x8,PQ8+16"
#IDMap,OPQ16_64,IMI2x8,PQ8+16"
USE_GPU = True

train_image_dir = r"D:/Kaggle/CleanCodeCup/2023/2023/images/images/"
index_path = "/faiss-web-service/resources/index"
ids_vectors_path = '/faiss-web-service/resources/ids_paths_vectors'

# ---------------------  Search
TOP_N = 5
SIMILARITY = 5


In [3]:
#FEATURE EXTRACTION
import numpy as np
import os
import cv2
import logging
import uuid
import sys

In [4]:
sys.path.append("..")
#from config import *

In [5]:
def calc_phash(gray_image):
  img = gray_image
  img = cv2.resize(img, (PHASH_X, PHASH_Y), interpolation=cv2.INTER_CUBIC)
  h, w = img.shape[:2]
  vis0 = np.zeros((h, w), np.float32)
  vis0[:h, :w] = img
  vis1 = cv2.dct(cv2.dct(vis0))
  vis1.resize(PHASH_X, PHASH_Y)
  img_list = vis1.flatten()
  avg = sum(img_list) * 1. / len(img_list)
  avg_list = [np.float32(0) if i < avg else np.float32(1) for i in img_list]
  return np.matrix(avg_list).flatten()

In [7]:
def adddPhash(gray_image, des):
  phash = calc_phash(gray_image)
  n, d = des.shape
  phash_mat = phash
  for i in range(n - 1):
    phash_mat = np.vstack((phash_mat, phash))
  des = np.hstack((des, phash_mat))
  return des

In [8]:

def calc_sift(sift, image_file):
  if not os.path.isfile(image_file):
    logging.error('Image:{} does not exist'.format(image_file))
    return -1, None
  try:
    image_o = cv2.imread(image_file)
  except:
    logging.error('Open Image:{} failed'.format(image_file))
    return -1, None
  if image_o is None:
    logging.error('Open Image:{} failed'.format(image_file))
    return -1, None
  image = cv2.resize(image_o, (NOR_X, NOR_Y))
  if image.ndim == 2:
    gray_image = image
  else:
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  kp, des = sift.detectAndCompute(gray_image, None)
  if isAddPhash:
    des = adddPhash(gray_image, des)
  sift_feature = np.matrix(des)
  return 0, sift_feature

In [9]:
def get_sift():
  return cv2.xfeatures2d.SIFT_create(nfeatures=NUM_FEATURES, nOctaveLayers=3, contrastThreshold=0.04, edgeThreshold=10, sigma=1.6)

In [10]:
def get_vectors(sift, image):
  return calc_sift(sift, image)

In [11]:
def iterate_files(_dir):
  result = []
  for root, dirs, files in os.walk(_dir, topdown=True):
    for fl in files:
      if fl.endswith("jpg") or fl.endswith("JPG"):
        result.append(os.path.join(root, fl))
  '''for files in os.walk(_dir, topdown=True):
    (x, y, z) = files
  for fl in z:
    if fl.endswith("jpg") or fl.endswith("JPG"):
      result.append(os.path.join(x, fl))'''
  return result

In [12]:
img_list = iterate_files('D:/Kaggle/CleanCodeCup/2023/2023/images/images')
len(img_list)

86640

In [13]:
files = os.walk('D:/Kaggle/CleanCodeCup/2023/2023/images/images', topdown=True)

In [14]:
for f in files:
  (x, y, z) = f

In [15]:
print(len(z))

86641


In [16]:
res1 = []
for fl in z:
  if fl.endswith("jpg") or fl.endswith("JPG"):
    res1.append(os.path.join(x, fl))

In [17]:
print(len(res1))

86640


In [18]:
pip install -U setuptools

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install faiss-gpu

  Using cached faiss-gpu-1.7.1.post2.tar.gz (40 kB)
  Running setup.py clean for faiss-cpu
  Running setup.py clean for faiss-cpu
Failed to build faiss-cpu faiss-cpu
Note: you may need to restart the kernel to use updated packages.


  ERROR: Command errored out with exit status 1:
   command: 'c:\Users\Sergey\AppData\Local\Programs\Python\Python39\python.exe' -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\Sergey\\AppData\\Local\\Temp\\pip-install-lqnjp1go\\faiss-gpu\\setup.py'"'"'; __file__='"'"'C:\\Users\\Sergey\\AppData\\Local\\Temp\\pip-install-lqnjp1go\\faiss-gpu\\setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d 'C:\Users\Sergey\AppData\Local\Temp\pip-wheel-2z9h38fg'
       cwd: C:\Users\Sergey\AppData\Local\Temp\pip-install-lqnjp1go\faiss-gpu\
  Complete output (8 lines):
  running bdist_wheel
  running build
  running build_py
  running build_ext
  building 'faiss._swigfaiss' extension
  swigging faiss\faiss\python\swigfaiss.i to faiss\faiss\python\swigfaiss_wrap.cpp
  swig.exe -python -c++ -Doverride= -I/usr/local/include -Ifaiss -DSWIGWIN -o fa

In [15]:
import faiss
faiss

<module 'faiss' from 'c:\\Users\\Sergey\\PycharmProjects\\Python3_9\\venv\\lib\\site-packages\\faiss\\__init__.py'>

In [8]:
#BUILD AND TRAIN INDEX
import numpy as np
import logging
import faiss
import pickle
import sys

In [9]:
dimensions = SIFT_DIMENSIONS
if isAddPhash:
  dimensions += PHASH_X * PHASH_Y
index = faiss.index_factory(dimensions, INDEX_KEY)
if USE_GPU:
  print("Use GPU...")
  res = faiss.StandardGpuResources()
  index = faiss.index_cpu_to_gpu(res, 0, index)
images_list = iterate_files(r'D:/Kaggle/CleanCodeCup/2023/2023/images/images')
ids_count = 0
index_dict = {}
ids = None
features = np.matrix([])
sift = get_sift()

Use GPU...


AttributeError: module 'faiss' has no attribute 'StandardGpuResources'

In [23]:
pip install --upgrade setuptools

Note: you may need to restart the kernel to use updated packages.


In [24]:
pip install faiss-gpu


  Using cached faiss-gpu-1.7.1.post2.tar.gz (40 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
Discarding https://files.pythonhosted.org/packages/17/76/47d0cc8161f4bf988583a2839bb1e56baf09d6b80cfa472b9eba4d5f543b/faiss-gpu-1.7.1.post2.tar.gz (from https://pypi.org/simple/faiss-gpu/): Requested faiss-cpu from https://files.pythonhosted.org/packages/17/76/47d0cc8161f4bf988583a2839bb1e56baf09d6b80cfa472b9eba4d5f543b/faiss-gpu-1.7.1.post2.tar.gz has inconsistent name: expected 'faiss-gpu', but metadata has 'faiss-cpu'
  Using cached faiss-gpu-1.7.1.post1.tar.gz (41 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
Discarding https://files.pythonhosted.org/packages/39/8d/b62bc92c8dd4b2a99d4a06b8804280f6445748b6d698eabb037e111080c7/faiss-gpu-1.7.1.post1.tar.gz (from https://pypi.org/simple/faiss-gpu/): Requested faiss-cpu from https://files.pythonhosted.org/packages/39/8d/b62bc9

  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [9 lines of output]
      c:\Users\Sergey\PycharmProjects\Python3_9\venv\lib\site-packages\setuptools\__init__.py:85: _DeprecatedInstaller: setuptools.installer and fetch_build_eggs are deprecated. Requirements should be satisfied by a PEP 517 installer. If you are using pip, you can try `pip install --use-pep517`.
        dist.fetch_build_eggs(dist.setup_requires)
      running egg_info
      creating C:\Users\Sergey\AppData\Local\Temp\pip-pip-egg-info-id3l02uy\faiss_cpu.egg-info
      writing C:\Users\Sergey\AppData\Local\Temp\pip-pip-egg-info-id3l02uy\faiss_cpu.egg-info\PKG-INFO
      writing dependency_links to C:\Users\Sergey\AppData\Local\Temp\pip-pip-egg-info-id3l02uy\faiss_cpu.egg-info\dependency_links.txt
      writing top-level names to C:\Users\Sergey\AppData\Local\Temp\pip-pip-egg-info-id3l02uy\faiss_cpu.egg-info\top_level.txt
      writing manifest file 'C

In [ ]:
for file_name in images_list:
  ret, sift_feature = calc_sift(sift, file_name)
  if ret == 0 and sift_feature.any():
    image_dict = {ids_count: (file_name, sift_feature)}
    index_dict.update(image_dict)
    #print(sift_feature.shape[0])
    ids_list = np.linspace(ids_count, ids_count, num=sift_feature.shape[0], dtype="int64")
    ids_count += 1
    if features.any():
      features = np.vstack((features, sift_feature))
      ids = np.hstack((ids, ids_list))
    else:
      features = sift_feature
      ids = ids_list
    if ids_count % 9000000 == 8999999:
      if not index.is_trained and INDEX_KEY != "IDMap,Flat":
        index.train(features)
      index.add_with_ids(features, ids)
      ids = None
      features = np.matrix([])
if features.any():
  if not index.is_trained and INDEX_KEY != "IDMap,Flat":
    index.train(features)
    print('done training')
  index.add_with_ids(features, ids)
  print('done adding')
#faiss.write_index(index, 'index_path')

done training
done adding


In [ ]:
index_dict[51]

('/content/caltech_short/031.car-tire/031_0070.jpg',
 matrix([[ 0.,  0.,  0., ...,  1.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [62., 26.,  1., ...,  3.,  0.,  0.],
         ...,
         [ 4.,  9.,  5., ...,  1.,  0.,  0.],
         [10.,  6., 22., ...,  0.,  0.,  0.],
         [13.,  1.,  0., ...,  6.,  7.,  1.]], dtype=float32))

In [ ]:
ids_vectors_path = 'ids_path_vectors'

In [ ]:
print(index.is_trained)
print(index.ntotal)

True
125982


In [ ]:
with open(ids_vectors_path, 'wb+') as f:
  try:
    pickle.dump(index_dict, f, True)
  except EnvironmentError as e:
    logging.error('Failed to save index file error:[{}]'.format(e))
    f.close()
  except RuntimeError:
    logging.error('Failed to save index file error:[{}]'.format(v))
f.close()

In [ ]:
from google.colab import files
files.download("ids_path_vectors")

----------------------------------------
Exception happened during processing of request from ('::ffff:127.0.0.1', 50322, 0, 0)
Traceback (most recent call last):
  File "/usr/lib/python3.6/socketserver.py", line 320, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/lib/python3.6/socketserver.py", line 351, in process_request
    self.finish_request(request, client_address)
  File "/usr/lib/python3.6/socketserver.py", line 364, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/lib/python3.6/socketserver.py", line 724, in __init__
    self.handle()
  File "/usr/lib/python3.6/http/server.py", line 418, in handle
    self.handle_one_request()
  File "/usr/lib/python3.6/http/server.py", line 406, in handle_one_request
    method()
  File "/usr/lib/python3.6/http/server.py", line 639, in do_GET
    self.copyfile(f, self.wfile)
  File "/usr/lib/python3.6/http/server.py", line 800, in copyfile
    shutil.copyfil

In [ ]:
id_to_vector = GET_FAISS_ID_TO_VECTOR('/content/ids_path_vectors')

In [ ]:
id_to_vector['/content/caltech_256/image_0001.jpg']

TypeError: ignored

In [ ]:
#SEARCH SIMILARS
import heapq
import numpy as np
import sys

In [ ]:
#sys.path.append("..")
#from utils.feature_detect import get_vectors, get_sift
#from config import NUM_FEATURES, SIFT_DIMENSIONS, SIMILARITY
from read_array_from_java import read_array

In [ ]:
class FaissIndex(object):
  def __init__(self, index, id_to_vector):
    assert index
    self.index = index
    self.id_to_vector = id_to_vector
    self.sift = get_sift()
  
  def search_by_ids(self, ids, k):
    vectors = [self.id_to_vector(id_)[1] for id_ in ids]
    results = self.__search__(ids, vectors, k + 1)
    return results

  def search_by_vectors(self, vectors, k):
    vectors = read_array(vectors, SIFT_DIMENSIONS)
    count = vectors.shape[0]
    vectors = np.vstack((vectors, vectors))
    vectors = vectors[0:count, :]
    print(vectors.shape)
    ids = [None]
    results = self.__search__(ids, [vectors], k)
    return results

  def search_by_image(self, image, k):
    ids = [None]
    ret, vectors = get_vectors(self.sift, image)
    print(ret)
    results = self.__search__(ids, [vectors], k)
    return results

  def __search__(self, ids, vectors, topN):
    def neighbor_dict_with_path(id_, file_path, score):
      return {'id': float(id_), 'file_path': file_path, 'score': score}
    def neighbor_dict(id_, score):
      return {'id': float(id_), 'score': score}
    def result_dict_str(id_, neighbors):
      return {'id': id_, 'neighbors': neighbors}
    results = []
    need_hit = SIMILARITY
    for id_, siftfeature in zip(ids, vectors):
      scores, neighbors = self.index.search(siftfeature, k=topN) if siftfeature.size > 0 else ([], [])
      print(neighbors)
      n, d = neighbors.shape
      result_dict = {}
      for i in range(n):
        l = np.unique(neighbors[i]).tolist()
        for r_id in l:
          if r_id != -1:
            score = result_dict.get(r_id, 0)
            score += 1
            result_dict[r_id] = score
      h = []
      for k in result_dict:
        v = result_dict[k]
        if v >= need_hit:
          if len(h) < topN:
            heapq.heappush(h, (v, k))
          else:
            heapq.heappushpop(h, (v, k))
      result_list = heapq.nlargest(topN, h, key=lambda x: x[0])
      neighbors_scores = []
      for e in result_list:
        confidence = e[0] * 100 / n
        if self.id_to_vector:
          print("A")
          file_path = self.id_to_vector(e[1])[0]
          neighbors_scores.append(neighbor_dict_with_path(e[1], file_path, str(confidence)))
        else:
          neighbors_scores.append(neighbor_dict(e[1], str(confidence)))
      results.append(result_dict_str(id_, neighbors_scores))
    return results

In [ ]:
fs2 = FaissIndex(index=index, id_to_vector=id_to_vector)

In [ ]:
fs2.search_by_image('/content/caltech_short/012.binoculars/012_0179.jpg', 3)

0
[[ 821  788 1036]
 [ 821  937  275]
 [ 821  828  552]
 [ 821  941   40]
 [ 821  900 1126]
 [ 821  936  391]
 [ 821  776  307]
 [ 821  674 1239]
 [ 821  496 1184]
 [ 821  731  905]
 [ 821  820  305]
 [ 821  120  134]
 [ 821  813  918]
 [ 821  200  274]
 [ 821  853  372]
 [ 821 1015  950]
 [ 821  959  992]
 [ 821  286  380]
 [ 821  414  387]
 [ 821  141  783]
 [ 821  920  182]
 [ 821 1065  821]
 [ 821  799  923]
 [ 821  764  821]
 [ 821  924  829]
 [ 821  236  276]
 [ 821  880  317]
 [ 821  850  692]
 [ 821  972  783]
 [ 821  836  924]
 [ 821 1151  774]
 [ 821  766  275]
 [ 821  219  275]
 [ 821    5  920]
 [ 821  838  821]
 [ 821  821  838]
 [ 821 1077  106]
 [ 821  941  941]
 [ 821 1264  937]
 [ 821  453  276]
 [ 821  143  368]
 [ 821  113  289]
 [ 821  276  276]
 [ 821  813  248]
 [ 821  229  504]
 [ 821  353  281]
 [ 821  920  783]
 [ 821  771  817]
 [ 821  358  372]
 [ 821  281  587]
 [ 821  832  805]
 [ 821  933  143]
 [ 821   26 1165]
 [ 821  280  936]
 [ 821  938  813]
 [ 821  

[{'id': None,
  'neighbors': [{'file_path': '/content/caltech_short/012.binoculars/012_0179.jpg',
    'id': 821.0,
    'score': '100.0'}]}]